In [1]:
# for deblur_table_sotus_ucase_hdf5_rare2k

In [2]:
import numpy as np
import pandas as pd
from biom import load_table
from gneiss.util import match

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from dsfdr import dsfdr
from dsfdr import simulation
from dsfdr import statistics
from dsfdr import transform

In [4]:
## load biom table

In [5]:
def convert_biom_to_pandas(table):
    otu_table = pd.DataFrame(np.array(table.matrix_data.todense()).T,
                             index=table.ids(axis='sample'),
                             columns=table.ids(axis='observation'))
    return otu_table

In [6]:
table = load_table('deblur_table_sotus_ucase_hdf5_rare2k.biom')
otu_table = convert_biom_to_pandas(table)

In [7]:
otu_table.head()

,TACGGAGGATGCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGTGCGTAGGCGGACTGTCAAGTCAGCGGTAAAATTGAGAGGCTCAACCTCTTCCCGCCGTTGAAACTGGTGGTCTTGAGTGGATGAGAAGTACGCGGAATGCGTGGT,TACGGAGGATGCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGTGCGTAGGCGGACAGTTAAGTCAGCGGTAAAATTGAGAGGCTCAACCTCTTCCCGCCGTTGAAACTGATTGTCTTGAGTGGGCGAGAAGTATGCGGAATGCGTGGT,TACGTAGGTGGCGAGCGTTATCCGGAATGATTGGGCGTAAAGGGTGCGCAGGCGGTCCTGCAAGTCTGGAGTGAAACGCATGAGCTCAACTCATGCATGGCTTTGGAAACTGGAGGACTGGAGAGCAGGAGAGGGCGGTGGAACTCCATG,TACGTAGGTGGCAAGCGTTATCCGGAATTATTGGGCGTAAAGAGGGAGCAGGCGGCAGCAAAGGTCTGTGGTGAAAGACTGAAGCTTAACTTCAGTAAGCCATAGAAACCGGGCAGCTAGAGTGCAGGAGAGGATCGTGGAATTCCATGT,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGGGCGCAGACGGCGATGCAAGCCAGGAGTGAAAGCCCGGGGCCCAACCCCGGGACTGCTCTTGGAACTGCGTGGCTGGAGTGCAGGAGGGGCAGGCGGAATTCCTGG,TACGGAGGATGCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGTGCGTAGGCGGTCCGTTAAGTCAGCGGTAAAATTGCGGGGCTCAACCCCGTCGAGCCGTTGAAACTGGCAGACTTGAGTTGGCGAGAAGTACGCGGAATGCGCGGT,TACGTATGGAGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAGCGTAGGCGGCCTGCCAAGTCTGATGTGAAATACCGGGGCTCAACCCCGGGGCTGCATTGGAAACTGGCAGGCTGGAGTGTCGGAGAGGCAGGCGGAATTCCTAG,TACGTAGGTGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGCGTGTAGGCGGGAAAGCAAGTCAGATGTGAAAACCATGGGCTCAACCTGTGGCCTGCATTTGAAACTGTTTTTCTTGAGTACTGGAGAGGCAGACGGAATTCCTAG,TACGGAGGATGCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGTGCGCAGGCGGACTCTCAAGTCAGCGGTCAAATCGCGGGGCTCAACCCCGTTCCGCCGTTGAAACTGGGAGCCTTGAGTGCGCGAGAAGTAGGCGGAATGCGTGGT,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAGCGTAGACGGTGATGCAAGTCTGAAGTGAAAGGCGGGGGCTCAACCCCCGGACTGCTTTGGAAACTGTATGACTGGAGTGCAGGAGAGGTAAGTGGAATTCCTAG,...,TACGGAGGATGCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGTGCGCAGGCGGACTCTCAAGTCAGCGGTCAAATCGCGGGGCTCAACCCCGTTCCGCCGTTGAAACTGGTGGTCTTGAGTGGATGAGAAGTACGCGGAATGCGTGGT,TATAAGACAGATGTGAAATCCCCGGGCTCAACCTGGGACCTGCATTTGTGACTGTATCGCTAGAGTACGGTAGAGGGGGATGGAATTCCGCGTGTAGCAGTGAAATGCGTAGATATGCGGAGGAACACCGATGGCGAAGGCAATCCCCTG,TACGTAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGCGTGCGCAGGCGGTATGTAAGACAGTTGTGAAATCCCCGGGCTCAACCTGGGAACTGCATCTGTGACTGCATAGCTAGAGTACGGTAGAGTGGGATGGAATTCCGCGT,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAGCGTAGACGGCCATGCAAGCCTGGAGTGAAAGCCCGGGGCCCAACCCCGGGACTGCTCTGGGAACTGTGCGGCTGGAGTGCGGGAGGGGCAGGCGGAATTCCTGG,TACGTAGGTGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGCGTGTAGGCGGGAAAGCAAGTCAGATGTGAAAACCATGGGCTCAACCTGTGGCCTGCATTTGAAACTGTTTTTCTTGAGTACTGGAGAGGCAGACGGACTTCCTAG,TACGGAGGATGCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGTGCGTAGGCGGACTGTCAAGTCAGCGGTCAAATCGCGGGGCTCAACCCCGTTCCGCCGTTGAAACTGGGAGCCTTGAGTGCGCGAGAAGTAGGCGGAATGCGTGGT,TACGTAGGTGGCAAGCGTTATCCGGAATTATTGGGCGTAAAGCGCGCGTAGGCGGTTTCTTAAGTCTGATGTGAAAGCCCACGGCTCAACCGTGGAGGGTCATTGGAAGCTGGGAAACTTGAGTGCAGAAGAGGAGAGTGGAATTCCATG,TACGTAGGGTGCGAGCGTTGTCCGGAATTATTGGGCGTAAAGAGCTTGTAGGCGGTTTGTCGCGTCTGCTGTGAAAGACCGGGGCGTAACTCCGGTTCTGCAGTGGGTACGGGCAGACTAGAGTGTGGTAGGGGAGACTGGAATTCCTGG,TACGTAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGTGCGCAGGCGGCTTTGCAAGACAGATGTGAAATCCCCGGGCTCAACCTGGGAGCTGCATTTGTGACTGCAAGGCTAGAGTACGGCAGAGGGGGATGGAATTCCGCG,TACGTAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGAGTGCGCAGGCGGTTTTGCAAGACCGATGTGAAATCCCCGGGCTTAACCTGGGAACTGCATTGGTGACTGCAAGGCTAGAGTGTGTCAGAGGGAGGTGGAATTCCGCA
10422.17.F.10,837.0,397.0,0.0,123.0,111.0,82.0,0.0,8.0,44.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10422.17.F.11,714.0,268.0,0.0,44.0,247.0,68.0,0.0,99.0,29.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10422.17.F.12,642.0,266.0,0.0,35.0,342.0,59.0,0.0,53.0,27.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10422.17.F.13,607.0,270.0,0.0,23.0,408.0,49.0,0.0,22.0,22.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10422.17.F.3,528.0,276.0,0.0,129.0,297.0,43.0,0.0,17.0,23.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
otu_table.shape

(182, 734)

In [9]:
## load mapping file

In [10]:
mapping = pd.read_table("haddad_6week_metadata_sampMatch_rare2k.txt", sep='\t', header=0, index_col=0)

In [11]:
mapping.shape

(182, 69)

In [12]:
mapping.head()

,BarcodeSequence,LinkerPrimerSequence,center_name,experiment_design_description,extraction_robot,extractionkit_lot,instrument_model,library_construction_protocol,linker,mastermix_lot,...,physical_specimen_location,physical_specimen_remaining,sample_type,scientific_name,sex,title,weekly_cage_food_consumption,weight,weight_units,Description
#SampleID,,,,,,,,,,,,,,,,,,,,,
10422.17.F.10,GTTGTTCTGGGA,GTGTGCCAGCMGCCGCGGTAA,UCSDMI,Mouse cohort exposed to apnea and controls to ...,HOWE_KF1,PM16B24,Illumina HiSeq 2500,"EMP 16S V4 protocol 515fbc, 806r",GT,14663,...,UCSD LBR -80 freezer,True,stool,mouse gut metagenome,male,OSA,Missing: Not provided,25.6,g,feces mouse 17 collection 10 of 13
10422.17.F.11,TGTGCTTGTAGG,GTGTGCCAGCMGCCGCGGTAA,UCSDMI,Mouse cohort exposed to apnea and controls to ...,HOWE_KF2,PM16B24,Illumina HiSeq 2500,"EMP 16S V4 protocol 515fbc, 806r",GT,14663,...,UCSD LBR -80 freezer,True,stool,mouse gut metagenome,male,OSA,71.8,25.2,g,feces mouse 17 collection 11 of 13
10422.17.F.12,AGAATCCACCAC,GTGTGCCAGCMGCCGCGGTAA,UCSDMI,Mouse cohort exposed to apnea and controls to ...,HOWE_KF1,PM16B24,Illumina HiSeq 2500,"EMP 16S V4 protocol 515fbc, 806r",GT,14663,...,UCSD LBR -80 freezer,True,stool,mouse gut metagenome,male,OSA,Missing: Not provided,25.7,g,feces mouse 17 collection 12 of 13
10422.17.F.13,CTGTAAAGGTTG,GTGTGCCAGCMGCCGCGGTAA,UCSDMI,Mouse cohort exposed to apnea and controls to ...,HOWE_KF2,PM16B24,Illumina HiSeq 2500,"EMP 16S V4 protocol 515fbc, 806r",GT,14663,...,UCSD LBR -80 freezer,True,stool,mouse gut metagenome,male,OSA,71.7,26.3,g,final feces mouse 17 collection 13 of 13
10422.17.F.3,CTCCCGAGCTCC,GTGTGCCAGCMGCCGCGGTAA,UCSDMI,Mouse cohort exposed to apnea and controls to ...,HOWE_KF2,PM16B24,Illumina HiSeq 2500,"EMP 16S V4 protocol 515fbc, 806r",GT,14663,...,UCSD LBR -80 freezer,True,stool,mouse gut metagenome,male,OSA,105.7,24.9,g,feces mouse 17 collection 3 of 13


In [13]:
mapping.exposure_type.value_counts()

IHH    92
Air    90
Name: exposure_type, dtype: int64

In [14]:
# match mapping file and biom table
mapping, otu_table = match(mapping, otu_table)

In [15]:
print(mapping.shape)
print(otu_table.shape)

(182, 69)
(182, 734)


In [16]:
# convert values in exposure_type to be integers
labels = np.array((mapping['exposure_type'] == 'IHH').astype(int))

In [17]:
labels

array([1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0,
       0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1])

In [18]:
# transpose otu_table to have OTUs as rows and samples as columns
dat = np.transpose(np.array(otu_table))

In [19]:
dat.shape

(734, 182)

In [20]:
# apply DS-FDR
result = dsfdr.dsfdr(dat, labels, transform_type='rankdata', method='meandiff', alpha=0.01, numperm=10000, fdr_method='dsfdr')

In [36]:
rej = result[0]

In [38]:
# total number of significant hypotheses
np.sum(rej)

82

In [39]:
s = pd.Series(rej, name='bools')

In [46]:
out = otu_table.T[s.values]

In [47]:
out.to_csv('deblur_dsfdr.csv', sep='\t')